In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
from scipy.sparse import vstack, hstack
from sklearn.neighbors import KNeighborsClassifier





In [ ]:
df = pd.read_csv("data/preprocessed_reviews.csv")
df

,Comment,Sentiment,comment_english,copy_comment,STREM+FPS3,STREM+FPS5,STREM+FPS7,STREM+ZEM
0,evet anlatıldığı gibi,1,evet anlatildigi,evet anlatıldığı,eve anl,evet anlat,evet anlatil,evet anla
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,oncede almistim cihazdan ense sakal tuketmek i...,öncede almıştım cihazdan ense sakal tüketmek n...,onc alm cih ens sak tuk ici num sif yak ali,onced almis cihaz ense sakal tuket icin numar ...,oncede almisti cihazda ense sakal tuketme icin...,önce al cihaz ense sakal tüket numara sıfır ya...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,urun basarili sakal kesmede baslik sayisi fazl...,ürün başarılı sakal kesmede başlık sayısı fazl...,uru bas sak kes bas say faz ola 0 yak ali kir ...,urun basar sakal kesme basli sayis fazla olabi...,urun basaril sakal kesmede baslik sayisi fazla...,ürün başarı sakal kes başlık sayı fazla olabil...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,oncede aynisini almistim cok guzel kaliteli ur...,öncede aynısını almıştım güzel kaliteli ürün .,onc ayn alm cok guz kal uru,onced aynis almis cok guzel kalit urun,oncede aynisin almisti cok guzel kalitel urun,önce aynı al güzel kalite ürün
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaforuyum ense sifir sakal trasi icin u...,erkek kuaförüyüm ense sıfır sakal traşı uygun ...,erk kua ens sif sak tra ici uyg uru,erkek kuafo ense sifir sakal trasi icin uygun ...,erkek kuaforu ense sifir sakal trasi icin uygu...,erkek kuaför ense sıfır sakal traş uygun ürün
...,...,...,...,...,...,...,...,...
15165,ışık seviyesi rezalet,0,isik seviyesi rezalet,ışık seviyesi rezalet,isi sev rez,isik seviy rezal,isik seviyes rezalet,ışık seviye rezalet
15166,Hic begenmedim. Aydinlatma hic yok ve her kapa...,0,hic begenmedim . aydinlatma hic kapattigimda a...,hic begenmedim . aydinlatma hic kapattigimda a...,hic beg ayd hic kap aya sif,hic begen aydin hic kapat ayari sifir,hic begenme aydinla hic kapatti ayari sifirla,hic begenmedim aydinlatma hic kapattigimda aya...
15167,2 gün sonra hoparlörü bozuldu kullanışsız,0,2 gun hoparloru bozuldu kullanissiz,2 gün hoparlörü bozuldu kullanışsız,2 gun hop boz kul,2 gun hopar bozul kulla,2 gun hoparlo bozuldu kullani,2 gün hoparlör boz kullan
15168,aşırı boğuk bir sesi ve rengi var kumanda heme...,0,asiri boguk sesi rengi kumanda hemen bozuldu .,aşırı boğuk sesi rengi kumanda hemen bozuldu .,asi bog ses ren kum hem boz,asiri boguk sesi rengi kuman hemen bozul,asiri boguk sesi rengi kumanda hemen bozuldu,aşırı boğuk ses renk kumanda hemen boz


# KNN with Our preprocessed data

<h1>KNN with 3 possible values

In [42]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=20000,
    min_df=3,
    max_df=0.9
)

X = tfidf.fit_transform(df["comment_english"])
y = df["Sentiment"]
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 182179 stored elements and shape (15170, 10948)>

In [43]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)


In [44]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    metric="cosine",
    weights="distance"
)

In [45]:
knn.fit(X_train, y_train)


KNeighborsClassifier(metric='cosine', weights='distance')

In [46]:
y_val_pred_knn = knn.predict(X_val)

print("k-NN – Validation results")
print(classification_report(y_val, y_val_pred_knn))
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Confusion matrix:")
print(confusion_matrix(y_val, y_val_pred_knn))

k-NN – Validation results
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      1396
           1       0.81      0.95      0.87      1360
           2       0.55      0.30      0.39       278

    accuracy                           0.84      3034
   macro avg       0.76      0.70      0.71      3034
weighted avg       0.84      0.84      0.83      3034

Accuracy: 0.8424522083058669
Confusion matrix:
[[1186  163   47]
 [  52 1287   21]
 [  56  139   83]]


In [47]:
y_test_pred_knn = knn.predict(X_test)

print("k-NN – Test results")
print(classification_report(y_test, y_test_pred_knn))
print("Accuracy:", accuracy_score(y_test, y_test_pred_knn))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_test_pred_knn))


k-NN – Test results
              precision    recall  f1-score   support

           0       0.92      0.87      0.89      1395
           1       0.82      0.96      0.89      1360
           2       0.62      0.29      0.40       279

    accuracy                           0.86      3034
   macro avg       0.79      0.71      0.73      3034
weighted avg       0.85      0.86      0.84      3034

Accuracy: 0.8553065260382333
Confusion matrix:
[[1213  146   36]
 [  46 1300   14]
 [  66  131   82]]


<h1>KNN with 2 possible values

In [48]:
df_v2 = df.loc[df["Sentiment"] != 2]

In [49]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=20000,
    min_df=3,
    max_df=0.9
)

X_v2 = tfidf.fit_transform(df_v2["comment_english"])
y_v2 = df_v2["Sentiment"]

In [50]:
X_temp_v2, X_test_v2, y_temp_v2, y_test_v2 = train_test_split(
    X_v2, y_v2, test_size=0.2, random_state=42, stratify=y_v2
)

X_train_v2, X_val_v2, y_train_v2, y_val_v2 = train_test_split(
    X_temp_v2, y_temp_v2, test_size=0.25, random_state=42, stratify=y_temp_v2
)

In [51]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    metric="cosine",
    weights="distance"
)

In [52]:
knn.fit(X_train_v2, y_train_v2)

KNeighborsClassifier(metric='cosine', weights='distance')

In [53]:
y_val_pred_knn_v2 = knn.predict(X_val_v2)

print("k-NN – Validation results")
print(classification_report(y_val_v2, y_val_pred_knn_v2))
print("Accuracy:", accuracy_score(y_val_v2, y_val_pred_knn_v2))
print("Confusion matrix:")
print(confusion_matrix(y_val_v2, y_val_pred_knn_v2))

k-NN – Validation results
              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1396
           1       0.89      0.96      0.92      1360

    accuracy                           0.92      2756
   macro avg       0.92      0.92      0.92      2756
weighted avg       0.92      0.92      0.92      2756

Accuracy: 0.9219883889695211
Confusion matrix:
[[1237  159]
 [  56 1304]]


In [54]:
y_test_pred_knn_v2 = knn.predict(X_test_v2)
print("k-NN – Test results")
print(classification_report(y_test_v2, y_test_pred_knn_v2))
print("Accuracy:", accuracy_score(y_test_v2, y_test_pred_knn_v2))
print("Confusion matrix:")
print(confusion_matrix(y_test_v2, y_test_pred_knn_v2))

k-NN – Test results
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      1396
           1       0.89      0.96      0.92      1360

    accuracy                           0.92      2756
   macro avg       0.92      0.92      0.92      2756
weighted avg       0.92      0.92      0.92      2756

Accuracy: 0.9212626995645864
Confusion matrix:
[[1232  164]
 [  53 1307]]


# KNN algorithm with Zemberek nlp method

In [56]:
df = df.dropna(subset=['STREM+ZEM'])

In [57]:
df["STREM+ZEM"].isna().sum()

np.int64(0)

In [58]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=20000,
    min_df=3,
    max_df=0.9
)

X = tfidf.fit_transform(df["STREM+ZEM"])
y = df["Sentiment"]
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 175583 stored elements and shape (15151, 9390)>

In [59]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)


In [60]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    metric="cosine",
    weights="distance"
)

In [61]:
knn.fit(X_train, y_train)


KNeighborsClassifier(metric='cosine', weights='distance')

In [62]:
y_val_pred_knn = knn.predict(X_val)

print("k-NN – Validation results")
print(classification_report(y_val, y_val_pred_knn))
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Confusion matrix:")
print(confusion_matrix(y_val, y_val_pred_knn))

k-NN – Validation results
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      1395
           1       0.78      0.92      0.84      1357
           2       0.59      0.27      0.37       278

    accuracy                           0.82      3030
   macro avg       0.75      0.67      0.69      3030
weighted avg       0.81      0.82      0.80      3030

Accuracy: 0.8158415841584158
Confusion matrix:
[[1153  210   32]
 [  92 1244   21]
 [  58  145   75]]


In [63]:
y_test_pred_knn = knn.predict(X_test)

print("k-NN – Test results")
print(classification_report(y_test, y_test_pred_knn))
print("Accuracy:", accuracy_score(y_test, y_test_pred_knn))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_test_pred_knn))


k-NN – Test results
              precision    recall  f1-score   support

           0       0.86      0.83      0.85      1395
           1       0.78      0.89      0.83      1357
           2       0.55      0.26      0.35       279

    accuracy                           0.80      3031
   macro avg       0.73      0.66      0.68      3031
weighted avg       0.80      0.80      0.79      3031

Accuracy: 0.8046849224678324
Confusion matrix:
[[1163  197   35]
 [ 129 1203   25]
 [  57  149   73]]


# KNN with 2 possible values

In [64]:
df_v2 = df.loc[df["Sentiment"] != 2]

In [65]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=20000,
    min_df=3,
    max_df=0.9
)

X_v2 = tfidf.fit_transform(df_v2["STREM+ZEM"])
y_v2 = df_v2["Sentiment"]

In [66]:
X_temp_v2, X_test_v2, y_temp_v2, y_test_v2 = train_test_split(
    X_v2, y_v2, test_size=0.2, random_state=42, stratify=y_v2
)

X_train_v2, X_val_v2, y_train_v2, y_val_v2 = train_test_split(
    X_temp_v2, y_temp_v2, test_size=0.25, random_state=42, stratify=y_temp_v2
)

In [67]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    metric="cosine",
    weights="distance"
)

In [68]:
knn.fit(X_train_v2, y_train_v2)

KNeighborsClassifier(metric='cosine', weights='distance')

In [69]:
y_val_pred_knn_v2 = knn.predict(X_val_v2)

print("k-NN – Validation results")
print(classification_report(y_val_v2, y_val_pred_knn_v2))
print("Accuracy:", accuracy_score(y_val_v2, y_val_pred_knn_v2))
print("Confusion matrix:")
print(confusion_matrix(y_val_v2, y_val_pred_knn_v2))

k-NN – Validation results
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1395
           1       0.84      0.92      0.88      1357

    accuracy                           0.88      2752
   macro avg       0.88      0.88      0.88      2752
weighted avg       0.88      0.88      0.88      2752

Accuracy: 0.8757267441860465
Confusion matrix:
[[1161  234]
 [ 108 1249]]


In [70]:
y_test_pred_knn_v2 = knn.predict(X_test_v2)
print("k-NN – Test results")
print(classification_report(y_test_v2, y_test_pred_knn_v2))
print("Accuracy:", accuracy_score(y_test_v2, y_test_pred_knn_v2))
print("Confusion matrix:")
print(confusion_matrix(y_test_v2, y_test_pred_knn_v2))

k-NN – Test results
              precision    recall  f1-score   support

           0       0.92      0.86      0.89      1395
           1       0.86      0.92      0.89      1357

    accuracy                           0.89      2752
   macro avg       0.89      0.89      0.89      2752
weighted avg       0.89      0.89      0.89      2752

Accuracy: 0.8880813953488372
Confusion matrix:
[[1198  197]
 [ 111 1246]]
